In [5]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from selenium.webdriver.common.by import By
from sys import platform
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
print(platform)

linux


!pip install bs4
!pip install selenium
!pip install fake-useragent
!pip install webdriver-manager
!pip install nltk

In [7]:
MonthDict={ "января" : 1,
      "февраля": 2,
       "марта": 3,
       "апреля": 4,
       "мая": 5,
       "июня": 6,
       "июля": 7,
       "августа": 8,
       "сентября": 9,
       "октября": 10,
       "ноября": 11,
       "декабря": 12
}

In [8]:
#обозначили сайт, проверили, чтобы 200 ок
page_link = 'https://www.facebook.com/groups/realtor.russia'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [39]:
#firefox_profile = webdriver.FirefoxProfile()
#firefox_profile.set_preference("intl.accept_languages", )
#firefox_profile.update_preferences()



options = webdriver.ChromeOptions()
#opts.add_argument("--headless")

#options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
options.add_argument("–lang= s")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(chrome_options=options, executable_path=ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/home/victor/.wdm/drivers/chromedriver/linux64/86.0.4240.22/chromedriver] found in cache


/home/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


#две версии вызова робобраузера, для винды и для линукса. 
#PS. В винде указан путь, который верен только для моей локалки
if 'win' in platform:
    from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = False
    options = Options()
    options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
    driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Users\matyukhinaa\Downloads\chromedriver_win32\chromedriver.exe", )
elif platform == 'linux':
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    

In [40]:
#вызов нужного сайта
driver.get(page_link)
time.sleep(6)
ok_button = driver.find_elements_by_xpath("//span[contains(text(), 'OK')]")
try:
    ok_button[0].click()
except:
    pass

In [41]:
#Количество скроллов вниз, чем больше, тем больше получим записей. Но дольше будет работать
times_scroll = 50

In [42]:
#Цикл для повторения прокрутки
for i in tqdm(range(times_scroll)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    if i%5==0: #Здесь каждую пятую прокрутку цикл засыпает на 1 секунду, чтобы страница прогрузилась
               #Если убрать эту строчку, может не дорабатывать до конца и прогружать меньше
        time.sleep(1)

100%|██████████| 50/50 [00:10<00:00,  4.61it/s]


In [43]:
#Костыли для открытия комментов
buttons = driver.find_elements_by_xpath("//div[contains(text(), 'Ещё')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue

#Костыли, созданные, чтобы открыть все длинные посты. К сожалению к двум циклам пришлось прибегнуть, так как
#часть кнопок не нажимаются с первого раза из-за того, что сайт недогрузился или не доскролился до нужной кнопки            
for i in range(2):       
    should_restart = True
    while should_restart:
        should_restart = False
        buttons = driver.find_elements_by_xpath("//span[contains(text(), 'Показать ещё')]")
        for i in tqdm(range(len(buttons))):
            try: 
                buttons[i].click()
            except:
                continue

buttons = driver.find_elements_by_xpath("//span[contains(text(), 'ответ')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue
'''
buttons = driver.find_elements_by_xpath("//span[contains(text(), 'ответов')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue
'''

100%|██████████| 118/118 [00:56<00:00,  2.11it/s]


'\nbuttons = driver.find_elements_by_xpath("//span[contains(text(), \'ответов\')]")\nfor i in tqdm(range(len(buttons))):\n    try: \n        buttons[i].click()\n    except:\n        continue\n'

In [24]:
# Чтение HTML Страницы
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [25]:
posts = (soup.findAll('div', attrs = {'class':"kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"}))
b = soup.findAll('span', attrs = {'class':"j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb gpro0wi8"})
#текст постов и время размещения *не спрашивайте почему b*

In [26]:
b

[<span aria-label="13 мая 2014 г." aria-labelledby="jsc_c_b" class="j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb gpro0wi8"><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4"><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t

In [27]:
#from datetime import timedelta
import datetime
import time

In [28]:
(datetime.datetime.now() - datetime.timedelta (days = 1)).day

15

In [29]:
#Загрузка сегодняшней даты в нужном формате
day_td = str(datetime.datetime.now().day)
day_ytd = str((datetime.datetime.now() - datetime.timedelta (days = 1)).day)
month_td = list(MonthDict.keys())[list(MonthDict.values()).index(datetime.datetime.now().month)]
year_td = str(datetime.datetime.now().year)
date_td = day_td+month_td+year_td
date_ytd = day_ytd+month_td+year_td
#Куча ИФ циклов для чистки и преобразования данных о времени
parsed_ls = []
#Первый цикл, для создания листа, где не будет лишних знаков
for i in tqdm(range(len(b))):
    parsed = (b[i].text.replace('=','').replace('·','').replace('\xa0', ''))
    if b[i].text != '':
        parsed_ls.append(parsed)
#Выкидывание ненужной информации и замена часов, минут и дней на дату
for i in tqdm(range(len(parsed_ls))):
    parsed_ls[i] = parsed_ls[i].replace(' ', '')
    parsed_ls[i] = parsed_ls[i].replace('г.Москва', '')
    parsed_ls[i] = parsed_ls[i].replace('г.', '')
    if 'ч.' in parsed_ls[i]:
        parsed_ls[i] = date_td
    if 'мин.' in parsed_ls[i]:
        parsed_ls[i] = date_td
    if 'Вчера' in parsed_ls[i]:
        parsed_ls[i] = date_ytd
#Перевод дат в формат д/м/г
for i in tqdm(range(len(parsed_ls))):
    if (parsed_ls[i][-3]==':')==True:
        parsed_ls[i]= parsed_ls[i][:-6]+year_td
    if parsed_ls[i][-4:-2]!='20':
        parsed_ls[i] = parsed_ls[i]+parsed_ls[i-1][-4:]
    if parsed_ls[i][:2].isnumeric():
        month = str(MonthDict[parsed_ls[i][2:-4]])
        date = (parsed_ls[i][:2])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year
    if parsed_ls[i][:2].isnumeric()==False:
        month = str(MonthDict[parsed_ls[i][1:-4]])
        date = (parsed_ls[i][:1])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year

100%|██████████| 76/76 [00:00<00:00, 283600.63it/s]


In [30]:
#перевод дат в формат datetime, перевод листа с хтмл кодом в текстовый лист
dt_ls = []
for date in (parsed_ls):
    dt_ls.append(datetime.datetime.strptime(date, '%d/%m/%Y'))
for i in range(len(posts)):
    posts[i] = posts[i].text

In [31]:
#Создание датафрейма
Dict = {'time': dt_ls[:len(posts)], 'post': posts}
df = pd.DataFrame.from_dict(Dict)

In [32]:
new_cl = 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql gk29lw5a a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb hrzyx87i jq4qci2q a3bd9o3v knj5qynh m9osqain'

In [33]:
#считывание комментариев и количества комментариев под каждым постом
comms = (soup.findAll('div', attrs = {'class':"tw6a2znq sj5x9vvc d1544ag0 cxgpxx05"}))
#comm_num_cl = 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql gk29lw5a a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb hrzyx87i jq4qci2q a3bd9o3v knj5qynh m9osqain'
comm_num = (soup.findAll('span', attrs = {'class':new_cl}))

In [34]:
#выкидываем кол-во репостов
comm_num_ls = []
for i in tqdm(range(len(comm_num))):
    if 'Комментарии' in comm_num[i].text:
        comm_num_ls.append(int(comm_num[i].text.replace('Комментарии: ', '')))
#почти всегда получается так, что посты без комментов технически сьедают комменты,
#а нижние посты остаются без них. Добьем лист нулями. Общая картина текстов не теряется
if len(comm_num_ls)<len(df):
    for i in range(len(df)-len(comm_num_ls)):
        comm_num_ls.append(0)

100%|██████████| 98/98 [00:00<00:00, 191717.25it/s]


In [35]:
#Добавляем колонку с кол-вом комментов
df['number_comments'] = np.array(comm_num_ls[:len(df)])

In [36]:
#токенизируем комменты, чтобы выкинуть мусор и убрать обращение по имени
#шумовые слова выкидываю сразу, чтобы не засорять датасет
tw = TweetTokenizer()
det = TreebankWordDetokenizer()
noise = stopwords.words('russian') + list(punctuation)
upnoise = [letter.upper() for letter in noise]
sum_noise = noise+upnoise+['.','»','«']
for i in tqdm(range(len(comms))):
    comms[i] = comms[i].text
    tokenized_example = (tw.tokenize(comms[i])[2:])
    #filtered_example = [word for word in tokenized_example if not word in noise]
    comms[i] = det.detokenize(tokenized_example)

100%|██████████| 1565/1565 [00:00<00:00, 8391.81it/s]


In [37]:
#разбиваем и клеим так, чтобы они были одним предложением. А также чтобы у каждого поста нужное кол-во
comms_ls = []
j = 0 
for i in tqdm(range(len(df))):
    comms_ls.append(comms[j:j+df.iloc[i,2]])
    j+=df.iloc[i,2]

#for i in range(len(comms_ls)):
 #   sentnc = ''
 #   for j in range(len(comms_ls[i])):
 #       sentnc += comms_ls[i][j]
  #      sentnc += ' '
 #   comms_ls[i] = sentnc
df['comment'] = comms_ls

100%|██████████| 67/67 [00:00<00:00, 27702.91it/s]


In [38]:
df

,time,post,number_comments,comment
0,2014-05-13,Эта группа создана для обмена профессиональной...,71,"[админы, такой вопрос . В вашей группе состоял..."
1,2020-11-13,Многодетную семью выгоняют из единственного жи...,6,[Ольга в чем грамотного? В оформлении документ...
2,2020-11-13,Коллеги!ищу для себя однокомнатную квартиру в ...,3,[Михайлова и как ОДНА пенсионерка получила от ...
3,2020-11-16,"Коллеги, нужен совет коллективного разума.Имее...",37,"[Таранцева это было 50 лет назад ), вообще про..."
4,2020-11-15,Здравствуйте. Помогите составить схему продажи...,5,"[Коняшин какие?, Коняшин С опекой тоже проблем..."
...,...,...,...,...
62,2020-10-28,"Коллеги, я хочу передать заявку: Помощь клиент...",0,[]
63,2020-10-14,Есть у нас тут специалисты по аренде загородны...,0,[]
64,2020-11-12,"Друзья , мне нужен СРОЧНО !!! завтра утром в п...",0,[]
65,2020-11-11,Проф. Александр ПанасюкДИСТАНЦИОННОЕ ОБУЧЕНИЕ ...,0,[]


In [102]:
df.to_csv('fb.csv')

In [104]:
df.to_excel('exfb.xlsx')